In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Reading the file
Book = pd.read_csv('Downloads\\bookRS.csv')

In [3]:
Book.head()

,Unnamed: 0,User.ID,Book.Title,Book.Rating
0,1,276726,Classical Mythology,5
1,2,276729,Clara Callan,3
2,3,276729,Decision in Normandy,6
3,4,276736,Flu: The Story of the Great Influenza Pandemic...,8
4,5,276737,The Mummies of Urumchi,6


In [4]:
Book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   10000 non-null  int64 
 1   User.ID      10000 non-null  int64 
 2   Book.Title   10000 non-null  object
 3   Book.Rating  10000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 312.6+ KB


In [5]:
Book.shape

(10000, 4)

In [6]:
Book.columns

Index(['Unnamed: 0', 'User.ID', 'Book.Title', 'Book.Rating'], dtype='object')

In [7]:
Book1 = Book.rename(columns={"User.ID": "ID","Book.Title":"Title","Book.Rating":"Rating"})

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
# Creating a Tfidf Vectorizer to remove all stop words
tfiBook = TfidfVectorizer(stop_words="english")  

In [10]:
# replacing the NaN values in overview column with
# empty string
Book1["Title"].isnull().sum() 

0

In [11]:
# Preparing the Tfidf matrix by fitting and transforming

tfiBook_matrix = tfiBook.fit_transform(Book1.Title)  
tfiBook_matrix.shape 

(10000, 11404)

In [12]:
from sklearn.metrics.pairwise import linear_kernel

# Computing the cosine similarity on TfiBook matrix
cosine_sim_matrix = linear_kernel(tfiBook_matrix,tfiBook_matrix)

In [13]:
# creating a mapping of book name to index number 
Book1_index = pd.Series(Book1.index,index=Book1['Title']).drop_duplicates()

In [14]:
print(Book1_index)

Title
Classical Mythology                                                                                      0
Clara Callan                                                                                             1
Decision in Normandy                                                                                     2
Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It       3
The Mummies of Urumchi                                                                                   4
                                                                                                      ... 
American Fried: Adventures of a Happy Eater.                                                          9995
Cannibal In Manhattan                                                                                 9996
How to Flirt: A Practical Guide                                                                       9997
Twilight                       

In [15]:
Book1_index["Classical Mythology"]

0

In [16]:
Book1_index["Tell Me This Isn't Happening"]

24

In [17]:
def get_Book_recommendations(Title,topN):  #topN = 10
    # Getting the book index using its Title 
    Book1_Title = Book1_index[Title]
        
    # Getting the pair wise similarity score for all the book's 
    cosine_scores = list(enumerate(cosine_sim_matrix[Book1_Title]))
    
    # Sorting the cosine_similarity scores based on scores 
    cosine_scores = sorted(cosine_scores,key=lambda x:x[1],reverse = True)
    
    # Get the scores of top 10 most similar book's 
    cosine_scores_10 = cosine_scores[0:topN+1]
    
    # Getting the book index 
    Book1_idx  =  [i[0] for i in cosine_scores_10]
    Book1_scores =  [i[1] for i in cosine_scores_10]
    
    # Similar movies and scores
    Book1_similar_read = pd.DataFrame(columns=["Title","Score"])
    Book1_similar_read["Title"] = Book1.loc[Book1_idx,"Title"]
    Book1_similar_read["Score"] = Book1_scores
    Book1_similar_read.reset_index(inplace=True)  
    Book1_similar_read.drop(["index"],axis=1,inplace=True)
    print (Book1_similar_read)
    

In [18]:
# Enter your Book and number of books to be recommended 
get_Book_recommendations("Tell Me This Isn't Happening", topN=10)

                                                Title     Score
0                        Tell Me This Isn't Happening  1.000000
1                                         Tell No One  0.486005
2                               And the Sea Will Tell  0.359438
3                                 Tell Me Your Dreams  0.353119
4   Prince Charming Isn't Coming: How Women Get Sm...  0.279408
5   The What's Happening to My Body? Book for Boys...  0.276636
6   KISS AND TELL (NANCY DREW FILES 104) : KISS AN...  0.270537
7              More Scary Stories To Tell In The Dark  0.249233
8                   Scary Stories to Tell In the Dark  0.249233
9                     Raised Catholic (Can You Tell?)  0.249189
10                       I Cannot Tell a Lie, Exactly  0.241275


In [19]:
# creating a mapping of book name to index number 
Book2_index = pd.Series(Book1.index,index=Book1['Rating']).drop_duplicates()

In [20]:
print(Book2_index)

Rating
5       0
3       1
6       2
8       3
6       4
     ... 
7    9995
9    9996
7    9997
8    9998
6    9999
Length: 10000, dtype: int64


In [21]:
Book2_index[10]

Rating
10       6
10      16
10      23
10      24
10      34
      ... 
10    9958
10    9967
10    9968
10    9969
10    9988
Length: 1732, dtype: int64

In [22]:
def get_Book_recommendations(Rating,topN):  #topN = 10
    # Getting the book index using its Title 
    Book1_Rating = Book1_index[Rating]
        
    # Getting the pair wise similarity score for all the book's 
    cosine_scores = list(enumerate(cosine_sim_matrix[Book1_Rating]))
    
    # Sorting the cosine_similarity scores based on scores 
    cosine_scores = sorted(cosine_scores,key=lambda x:x[1],reverse = True)
    
    # Get the scores of top 10 most similar book's 
    cosine_scores_10 = cosine_scores[0:topN+1]
    
    # Getting the book index 
    Book1_idx  =  [i[0] for i in cosine_scores_10]
    Book1_scores =  [i[1] for i in cosine_scores_10]
    
    # Similar movies and scores
    Book1_similar_read = pd.DataFrame(columns=["Rating","Score"])
    Book1_similar_read["Rating"] = Book1.loc[Book1_idx,"Rating"]
    Book1_similar_read["Score"] = Book1_scores
    Book1_similar_read.reset_index(inplace=True)  
    Book1_similar_read.drop(["index"],axis=1,inplace=True)
    print (Book1_similar_read)
    

In [23]:
Book2_index[10]

Rating
10       6
10      16
10      23
10      24
10      34
      ... 
10    9958
10    9967
10    9968
10    9969
10    9988
Length: 1732, dtype: int64

In [24]:
# Enter your Book and number of books to be recommended 
get_Book_recommendations(10, topN=20)

    Rating     Score
0        7  1.000000
1        8  0.618913
2        4  0.412809
3       10  0.389121
4        8  0.389121
5        6  0.364646
6       10  0.364380
7        8  0.364380
8        6  0.334253
9        7  0.315843
10       7  0.314089
11       3  0.314089
12       5  0.314089
13      10  0.311434
14      10  0.299105
15       7  0.289712
16       6  0.288989
17       7  0.287768
18       8  0.273958
19       5  0.264397
20      10  0.246087


Book Recommendation with pivot tool method

In [25]:
Book1.head()

,Unnamed: 0,ID,Title,Rating
0,1,276726,Classical Mythology,5
1,2,276729,Clara Callan,3
2,3,276729,Decision in Normandy,6
3,4,276736,Flu: The Story of the Great Influenza Pandemic...,8
4,5,276737,The Mummies of Urumchi,6


In [26]:
# Convert Title, Id and rating to numeric
Book1[['ID','Rating']] = Book1[['ID','Rating']].apply(pd.to_numeric)

In [27]:
# Convert table to pivot table
Book1_pivot = Book1.pivot_table(values = 'Rating', index = 'Title', columns = 'ID')  

In [28]:
# Replace NaNs with 0s
Book1_pivot.fillna(0, inplace = True)

In [29]:
# Display first few rows and columns of books and ratings pivot table
Book1_pivot.head()

ID,8,9,10,12,14,16,17,19,22,26,...,278831,278832,278836,278843,278844,278846,278849,278851,278852,278854
Title,,,,,,,,,,,,,,,,,,,,,
"Jason, Madison &amp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Other Stories;Merril;1985;McClelland &amp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Repairing PC Drives &amp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'O Au No Keia: Voices from Hawai'I's Mahu and Transgender Communities,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Extract and save book titles
Book1_titles = Book1_pivot.index

In [31]:
# Define correlation matrix to get correlation between books
corr_matrix = np.corrcoef(Book1_pivot)

In [32]:
# Display correlation matrix
corr_matrix

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
        -4.58505273e-04, -4.58505273e-04, -4.58505273e-04],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
        -4.58505273e-04, -4.58505273e-04, -4.58505273e-04],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
        -4.58505273e-04, -4.58505273e-04, -4.58505273e-04],
       ...,
       [-4.58505273e-04, -4.58505273e-04, -4.58505273e-04, ...,
         1.00000000e+00, -4.58505273e-04, -4.58505273e-04],
       [-4.58505273e-04, -4.58505273e-04, -4.58505273e-04, ...,
        -4.58505273e-04,  1.00000000e+00, -4.58505273e-04],
       [-4.58505273e-04, -4.58505273e-04, -4.58505273e-04, ...,
        -4.58505273e-04, -4.58505273e-04,  1.00000000e+00]])

In [33]:
# Define function provide top recommendations
def recommender(book_names):  
    # Initialize recommendations output 
    recommendations = np.zeros(corr_matrix.shape[0])
    # Loop through the book titles that the user has rated
    for book in book_names:
        # Sum the correlations
        recommendations = recommendations + corr_matrix[list(Book1_titles).index(book)]
    # Convert recommendations into dataframe
    recommendations_df = pd.DataFrame({
            'Title': Book1_titles,
            'Recommendation': recommendations})
    # Remove booke titles that the user has already rated
    recommendations_df = recommendations_df[~(recommendations_df.Title.isin(book_names))]
    # Sort from most correlated to least correlated
    recommendations_df = recommendations_df.sort_values(by=['Recommendation'], ascending = False)
    return recommendations_df

In [34]:
# Defining user
user = 2799

# Getting list of books the defined user has rated
user_reviews = Book1[Book1.ID == user].Title.tolist()  
print('Books rated by user\n')
print(user_reviews)

Books rated by user

['Carrot Tops and Cottontails', "We're Going on a Bear Hunt", 'The Elephant Tree', 'Hoot', 'Dinosaurs and All That Rubbish (Storytime Giants)', "Miss Bilberry's New House", 'Thomas and the Station Cat: Lift the Flap Book', "Humphrey's Corner", 'The Golden Egg', 'Spot Visits His Grandparents (Picture Puffins)', 'LA Machine Infernale (Le Livre de poche ; 854)', 'La Maison du sommeil']


In [35]:
# Getting the list of recommendations
recommendations = recommender(user_reviews)

# Print out top 10 recommendations
print('\nRecommendations for user\n')
print(recommendations.Title.head(10))


Recommendations for user

941                          Black Beauty
5171                    One Small Sparrow
8512          The Third Miracle : A Novel
321                     A Wrinkle In Time
6121              Shades of Jade: A Novel
9378             Where the Red Fern Grows
8631    The Vision of Emma Blau : A Novel
3958                         LITTLE BIRDS
2148                           Douce nuit
3419         I Cannot Tell a Lie, Exactly
Name: Title, dtype: object
